In [2]:
import pandas as pd
import numpy as np
import kagglehub
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor, KDTree
from sklearn.metrics import accuracy_score, mean_squared_error
import time

# --- Dataset Access (Only Download Code) ---
path = kagglehub.dataset_download("gregorut/videogamesales")
print(f"Path to dataset files: {path}")

# Load the dataset
# Assuming the file is named 'vgsales.csv' within the downloaded path
df = pd.read_csv(f"{path}/vgsales.csv")

# --- Data Preparation (Guidelines) ---

# 1. Drop rows with missing Year, Genre, Platform, or Global_Sales
cols_to_check = ['Year', 'Genre', 'Platform', 'Global_Sales']
df_clean = df.dropna(subset=cols_to_check).copy()

# 2. Convert Year to integer and optionally clip (1980-2020)
df_clean['Year'] = pd.to_numeric(df_clean['Year'], errors='coerce').astype('Int64')
df_clean['Year'] = df_clean['Year'].clip(lower=1980, upper=2020)

# 3. Define features (X) and targets (y)
# Drop Name and Rank as they are usually not predictive features
features = ['Year', 'Genre', 'Platform', 'Publisher', 'NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales']
X = df_clean[features]

# Global_Sales is the sum of regional sales, so we'll use regional sales as features
# to avoid leakage in the regressor, as specified in the objective.
# For the classifier, we use the original feature set (Year, Genre, Platform, etc.)

# Separate data for Classifier (Genre prediction) and Regressor (Global_Sales prediction)
# Classifier Target
y_genre = df_clean['Genre']
# Regressor Target
y_sales = df_clean['Global_Sales']

# --- Preprocessing Pipeline Setup ---

# Define numeric and categorical features
numeric_features = ['Year', 'NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales']
# Platform, Publisher and Genre for one-hot encoding
categorical_features = ['Platform', 'Publisher', 'Genre']

# Create preprocessor using ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        # 4. Use one-hot encoding for categorical features
        ('onehot', OneHotEncoder(handle_unknown='ignore'), categorical_features),
        # 5. Scale numeric features
        ('scaler', StandardScaler(), numeric_features)
    ],
    # Pass through other columns (not needed here since all are in one of the lists)
    remainder='passthrough'
)

# Apply preprocessing to the full feature set
X_processed = preprocessor.fit_transform(X)
# Get feature names after one-hot encoding for analysis if needed
feature_names = list(preprocessor.named_transformers_['onehot'].get_feature_names_out(categorical_features)) + numeric_features

# Convert to DataFrame for easier inspection (optional)
X_processed_df = pd.DataFrame(X_processed.toarray(), columns=feature_names, index=df_clean.index)
print("--- Data Preparation Complete ---")
print(f"Features shape: {X_processed.shape}")

Using Colab cache for faster access to the 'videogamesales' dataset.
Path to dataset files: /kaggle/input/videogamesales
--- Data Preparation Complete ---
Features shape: (16327, 625)


In [3]:
## 1) Build and evaluate a KNN classifier for Genre
print("\n--- KNN Classifier for Genre ---")

# Split data (using the cleaned data and Genre target)
X_train_cls, X_test_cls, y_train_cls, y_test_cls = train_test_split(
    X_processed, y_genre, test_size=0.3, random_state=42, stratify=y_genre
)

# Define hyperparameters for initial model (k=5, metric=Euclidean)
k_cls = 5
metric_cls = 'euclidean'

# Build and train the KNN Classifier model
knn_classifier = KNeighborsClassifier(n_neighbors=k_cls, metric=metric_cls)
knn_classifier.fit(X_train_cls, y_train_cls)

# Predict and evaluate
y_pred_cls = knn_classifier.predict(X_test_cls)
accuracy_cls = accuracy_score(y_test_cls, y_pred_cls)

print(f"Classifier (k={k_cls}, metric='{metric_cls}') Accuracy: **{accuracy_cls:.4f}**")


--- KNN Classifier for Genre ---
Classifier (k=5, metric='euclidean') Accuracy: **0.9588**


In [4]:
## 2) Build and evaluate a KNN regressor for Global_Sales (avoiding leakage)
print("\n--- KNN Regressor for Global_Sales (Leakage Avoidance) ---")

# --- Revised Preprocessing for Regressor to Avoid Leakage ---
# Features for Regressor: Only Year, Platform, Genre, Publisher (No regional sales)
# We need to re-run the preprocessor without regional sales columns for X_regressor
regressor_features = ['Year', 'Genre', 'Platform', 'Publisher']
X_regressor_raw = df_clean[regressor_features]

regressor_preprocessor = ColumnTransformer(
    transformers=[
        ('onehot', OneHotEncoder(handle_unknown='ignore'), ['Platform', 'Publisher', 'Genre']),
        ('scaler', StandardScaler(), ['Year'])
    ],
    remainder='passthrough'
)

X_regressor = regressor_preprocessor.fit_transform(X_regressor_raw)

# Split data (using the leakage-avoiding features and Global_Sales target)
X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(
    X_regressor, y_sales, test_size=0.3, random_state=42
)

# Define hyperparameters for initial model (k=5, metric=Manhattan)
k_reg = 5
metric_reg = 'manhattan' # L1 norm often good for regression

# Build and train the KNN Regressor model
knn_regressor = KNeighborsRegressor(n_neighbors=k_reg, metric=metric_reg)
knn_regressor.fit(X_train_reg, y_train_reg)

# Predict and evaluate
y_pred_reg = knn_regressor.predict(X_test_reg)
mse_reg = mean_squared_error(y_test_reg, y_pred_reg)
rmse_reg = np.sqrt(mse_reg)

print(f"Regressor (k={k_reg}, metric='{metric_reg}') RMSE: **{rmse_reg:.4f}** (in millions of dollars)")
print("> Lower RMSE is better. Note: This result is based on non-sales features to avoid leakage.")


--- KNN Regressor for Global_Sales (Leakage Avoidance) ---
Regressor (k=5, metric='manhattan') RMSE: **1.7843** (in millions of dollars)
> Lower RMSE is better. Note: This result is based on non-sales features to avoid leakage.


In [5]:
## 3) Construct a KD-tree
print("\n--- KD-Tree Construction and Nearest Neighbor Query ---")

# KD-Trees work best on data with fewer dimensions and Euclidean distance.
# We'll use the classifier's scaled feature set (X_processed).
# The KDTree class from sklearn.neighbors can be used directly.
kdtree = KDTree(X_processed.toarray())
print("KD-Tree built successfully on the full preprocessed dataset.")

# Example query: Find the 3 nearest neighbors to the first data point
query_point = X_processed[0].toarray().reshape(1, -1)
distances, indices = kdtree.query(query_point, k=3)

print(f"Query point index: {df_clean.index[0]}")
print(f"Nearest neighbor indices: {df_clean.index[indices[0]]}")
print(f"Distances: {distances[0]}")


## 4) Compare neighbor search backends (kd_tree, ball_tree, and brute)
print("\n--- Backend Comparison (Runtime) ---")

k_search = 10
n_queries = 100 # Number of queries to average the search time

# Generate a small set of random queries from the test set
np.random.seed(42)
query_indices = np.random.choice(X_test_cls.shape[0], n_queries, replace=False)
queries = X_test_cls[query_indices]

backends = ['brute', 'kd_tree', 'ball_tree']
runtimes = {}

for backend in backends:
    # Use the classifier setup, but enforce the algorithm
    knn_model = KNeighborsClassifier(n_neighbors=k_search, algorithm=backend)
    knn_model.fit(X_train_cls, y_train_cls)

    start_time = time.time()
    for q in queries:
        knn_model.kneighbors(q.reshape(1, -1))
    end_time = time.time()

    avg_time = (end_time - start_time) / n_queries
    runtimes[backend] = avg_time

print(pd.DataFrame({'Avg Query Time (s)': runtimes}).T)

# Analysis:
print("\n**Analysis of Backend Runtimes:**")
if runtimes['brute'] == min(runtimes.values()):
    print("Brute force was fastest. This often happens on datasets with **high dimensionality** or when the total number of data points is **small/medium**.")
elif runtimes['kd_tree'] == min(runtimes.values()):
    print("KD-Tree was fastest. This usually indicates the data is well-suited for partitioning (i.e., **low-to-moderate dimensionality**).")
elif runtimes['ball_tree'] == min(runtimes.values()):
    print("Ball-Tree was fastest. This is common with **high-dimensional data** or when using **non-Euclidean metrics**.")


## 5) Provide a concise analysis of hyperparameters (k, metric p in {1,2}), feature scaling, and dimensionality effects.
print("\n--- 5) Concise Analysis ---")

print("### Hyperparameters (k, metric p)")
print("* **k (Number of Neighbors):** Increasing *k* generally makes the model **smoother** and less sensitive to noise, reducing variance but potentially increasing bias (underfitting). A smaller *k* is more sensitive to local data, resulting in higher variance.")
print("* **Metric p $\in \{1, 2\}$:**")
print("    * **$p=1$ (Manhattan Distance):** Measures distance by the sum of absolute differences (L1 norm). It's more **robust to outliers** and often preferred in high-dimensional spaces or for regression.")
print("    * **$p=2$ (Euclidean Distance):** Measures the shortest straight-line distance (L2 norm). It's the standard for spatial data and is the default for many KNN implementations.")

print("\n### Feature Scaling")
print("* **Effect:** Scaling is **critical** for KNN. Since KNN relies on distance calculation (like Euclidean or Manhattan), features with a larger magnitude (e.g., Global_Sales) will **dominate** the distance function, regardless of their actual importance. `StandardScaler` (Z-score normalization) ensures all features contribute equally to the distance calculation.")

print("\n### Dimensionality Effects (The Curse of Dimensionality)")
print("* **Effect:** In high-dimensional spaces (which the one-hot encoding creates), the distance between any two points tends to become **almost equal**. This means that the concept of 'nearest' neighbor loses its meaning.")
print("* **Consequence:** Algorithms like **KD-Tree** become less efficient, often defaulting to performance close to **Brute Force** search. **Ball-Tree** is generally more robust to higher dimensions. For effective KNN in high dimensions, dimensionality reduction (like PCA) may be necessary."
)

<>:63: SyntaxWarning: invalid escape sequence '\i'
<>:63: SyntaxWarning: invalid escape sequence '\i'
/tmp/ipython-input-3196360621.py:63: SyntaxWarning: invalid escape sequence '\i'
  print("* **Metric p $\in \{1, 2\}$:**")



--- KD-Tree Construction and Nearest Neighbor Query ---
KD-Tree built successfully on the full preprocessed dataset.
Query point index: 0
Nearest neighbor indices: Index([0, 2, 3], dtype='int64')
Distances: [ 0.         52.13631851 55.46657722]

--- Backend Comparison (Runtime) ---


/usr/local/lib/python3.12/dist-packages/sklearn/neighbors/_base.py:598: UserWarning: cannot use tree with sparse input: using brute force
  warnings.warn("cannot use tree with sparse input: using brute force")
/usr/local/lib/python3.12/dist-packages/sklearn/neighbors/_base.py:598: UserWarning: cannot use tree with sparse input: using brute force
  warnings.warn("cannot use tree with sparse input: using brute force")


                       brute  kd_tree  ball_tree
Avg Query Time (s)  0.001293  0.00185   0.001808

**Analysis of Backend Runtimes:**
Brute force was fastest. This often happens on datasets with **high dimensionality** or when the total number of data points is **small/medium**.

--- 5) Concise Analysis ---
### Hyperparameters (k, metric p)
* **k (Number of Neighbors):** Increasing *k* generally makes the model **smoother** and less sensitive to noise, reducing variance but potentially increasing bias (underfitting). A smaller *k* is more sensitive to local data, resulting in higher variance.
* **Metric p $\in \{1, 2\}$:**
    * **$p=1$ (Manhattan Distance):** Measures distance by the sum of absolute differences (L1 norm). It's more **robust to outliers** and often preferred in high-dimensional spaces or for regression.
    * **$p=2$ (Euclidean Distance):** Measures the shortest straight-line distance (L2 norm). It's the standard for spatial data and is the default for many KNN implemen